In [1]:
%cd ..

/tf/notebooks


In [59]:
import json
import re
import random
from tqdm import tqdm_notebook as tqdm

In [3]:
with open("data/fddb_data_v3_withimg.json", encoding='utf-8') as f:
    fddb = json.load(f)

In [4]:
with open("data/recipe-ingredient-to-fooddb.json") as f:
    all_ings_orig = json.load(f)

In [5]:
from extract_ingredients.util import normalize_out_ingredient


In [16]:
custom_amounts = {("Öl", "Olivenöl"): ["UNK (20 ml)", "etwas (20 ml)"], # 2EL = 20ml
                  ("Salz",): ["UNK (2 g)"],
                 }

custom_amounts = {ks: v for k, v in custom_amounts.items() for ks in k}
custom_amounts

{'Olivenöl': ['UNK (20 ml)', 'etwas (20 ml)'],
 'Salz': ['UNK (2 g)'],
 'Öl': ['UNK (20 ml)', 'etwas (20 ml)']}

In [19]:
def normalize_amount_unit(unit):
    if re.match(r"^mittelgroße[sre]?$", unit):
        return "mittelgroß"
    if unit == "Dose (Abtropfgewicht)":
        return "Dose"
    return unit

In [20]:
from collections import defaultdict
fddb_entries = defaultdict(list)
rem = re.compile(r"^(\d+ )?(.*) \((\d+(?:,\d+)?) (g|ml)\)$")
def handle_fddb():
    for e in tqdm(fddb):
        for ning in normalize_out_ingredient(e["name"]):
            fddb_entries[ning].append(e)
        sanity = list(e["Standard Nährwerte"].keys())
        snmatch = re.match("^Nährwerte für 100 (\w+)$", sanity[0])
        if snmatch is not None:
            unn = snmatch.groups()[0]
            e["parsed"] = {"unit": unn, "for_100": e["Standard Nährwerte"]["Nährwerte für 100 {}".format(unn)]}

        else:
            print("SANITY ERROR:", e["Id"], sanity, unit)
            continue
        amts = e["parsed"]["amounts"] = []
        spec_keys = list(e["Spezifische Nährwerte"].keys())
        if e["name"] in custom_amounts:
            spec_keys += custom_amounts[e["name"]]
        for k in spec_keys:
            if k == "100 g (100 g)" or k == "100 g (100 ml)" or k == "100 ml (100 ml)":
                continue
            txtamount, txt, amount, unit = rem.match(k).groups()
            amount = float(amount.replace(",", "."))
            txtamount = float(txtamount.replace(",", ".")) if txtamount else 1.0
            txt = normalize_amount_unit(txt)
            amts.append((txt, k, amount/txtamount))

            if unn != unit:
                raise Exception("noooo " + unn + " " + unit, e["name"])
            #print(txt, amount/txtamount, unit)
handle_fddb()

 12%|█▏        | 14149/122977 [00:00<00:01, 71255.94it/s]

SANITY ERROR: https://fddb.info/db/de/lebensmittel/selbst_gemacht_100_kalorien_dummy/index.html ['Nährwerte deines Gerichts kennst und in dein Ernährungstagebuch eintragen möchtest.\n31 Bewertungen\nSchreibe eine Bewertung\nHersteller: Selbst gemacht\nProduktgruppe: Sonstiges\nDatenquelle: Schätzung. Die Produktdaten wurden am 18.05.2006 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 08.11.2018. '] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/weider_weider_yippie_riegel_cookies_netto_naehrwerte/index.html ['Nährwerte) von MarieCarlotta | Hochgeladen von: MarieCarlotta (Problem melden) Details\n\nFoto hochladen\nNoch keine Beschreibung für dieses Produkt.\nAngaben noch nicht bestätigt. Melde einen Fehler oder korrigiere die Angaben.\nSchreibe eine Bewertung\nHersteller: Weider\nProduktgruppe: Nahrungsergänzung\nDatenquelle: Extern. Die Produktdaten wurden am 09.11.2017 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 04.07.20

 29%|██▉       | 35776/122977 [00:00<00:01, 72051.43it/s]

SANITY ERROR: https://fddb.info/db/de/lebensmittel/imbiss_doener_kebap/index.html ['Nährwerte beziehen sich auf einen Döner mit Kalbfleisch.\n15 Bewertungen\nSchreibe eine Bewertung\nHersteller: Imbiss\nProduktgruppe: Fast Food\nDatenquelle: Extern. Die Produktdaten wurden am 25.04.2008 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 14.04.2015. \n\n\nNährwerte für 100 g'] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/maggi_magic_asia_gebratene_nudeln_india/index.html ['Nährwerte zubereitet inkl. Öl (Angaben laut Packung)\r\n\r\n\nDie Angaben beziehen sich auf das zubereitete Produkt.\nSchreibe eine Bewertung\nHersteller: Maggi\nProduktgruppe: Fertiggerichte\nDatenquelle: Extern. Die Produktdaten wurden am 08.07.2009 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 04.07.2018. EAN: 7613032277710 \n\n\nNährwerte für 100 g'] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/naturwohl_pharma_yokebe_classic_naehrwerte_pulver_ne

 53%|█████▎    | 64983/122977 [00:00<00:00, 73263.41it/s]

SANITY ERROR: https://fddb.info/db/de/lebensmittel/wls_products_crispy_rood_fruit_naehrwerte_pro_riegel/index.html ['Nährwerte pro riegel von waldvolk | Hochgeladen von: waldvolk (Problem melden) Details\n\nFoto hochladen\nNoch keine Beschreibung für dieses Produkt.\nAngaben noch nicht bestätigt. Melde einen Fehler oder korrigiere die Angaben.\nSchreibe eine Bewertung\nHersteller: WLS Products\nProduktgruppe: Neue Produkte\nDatenquelle: Extern. Die Produktdaten wurden am 04.04.2019 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 04.04.2019. \n\n\nNährwerte für 100 g'] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/naturprodukt_banane_roh/index.html [''] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/lidl_roesti_kartoffel-ecken/index.html ['Nährwerte beziehen sich auf das tiefgefrorene Produkt.\r\n\nSchreibe eine Bewertung\nHersteller: Lidl, Tenery\nProduktgruppe: Kartoffelprodukte\nDatenquelle: Extern. Die Produktdaten wurden am 08.10.2006 von ein

 77%|███████▋  | 94088/122977 [00:01<00:00, 72287.09it/s]

SANITY ERROR: https://fddb.info/db/de/lebensmittel/imbiss_doener_mit_haehnchenfleisch/index.html ['Nährwerte beziehen sich auf einen Döner mit Hähnchenfleisch (Chicken Döner).\n31 Bewertungen\nSchreibe eine Bewertung\nHersteller: Imbiss\nProduktgruppe: Fast Food\nDatenquelle: Schätzung, Rezeptur, Intern. Die Produktdaten wurden am 09.06.2008 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 12.12.2017. \n\n\nNährwerte für 100 g'] ml
SANITY ERROR: https://fddb.info/db/de/lebensmittel/durchschnittswert_kohlenhydrate_dummy/index.html [''] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/natur_pur_bio_kokosoel/index.html ['Nährwerte | Hochgeladen von: 84nk41 (Problem melden) Details\n\nNoch keine Beschreibung für dieses Produkt.\nBeinhaltet viel FettAngaben noch nicht bestätigt. Melde einen Fehler oder korrigiere die Angaben.Vegetarisch (vegan)\nSchreibe eine Bewertung\nHersteller: Natur Pur\nProduktgruppe: Neue Produkte\nDatenquelle: Extern. Die Produktdaten w

100%|██████████| 122977/122977 [00:01<00:00, 72236.71it/s]

SANITY ERROR: https://fddb.info/db/de/lebensmittel/naturprodukt_mandeln_roh/index.html [''] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/naturprodukt_apfel_frisch/index.html [''] g


In [21]:
# get recipe meta
with open("data/recipes/processed_data.json", encoding="utf-8") as f:
    recipes = json.load(f)

In [39]:
# get amount map from parse-amounts
with open("data/recipes/parsed_amounts.json") as f:
    parsed_amounts = json.load(f)

In [40]:
# only recipes with pictures
recipes = [recipe for recipe in recipes if len(recipe['picture_files']) > 0]

In [62]:
from operator import itemgetter
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from IPython.display import HTML
from html import escape
# ugly af code
debug=False
    
def match_amts(*, ing_matches, have_count, have_unit):
    possible_entries = {entry["Id"]: (ing_match[1], entry) for ing_match in ing_matches for entry in fddb_entries[ing_match[0]]}
    possible_entries = sorted(list(possible_entries.values()), key=lambda e: -e[0])
    have_unit = [normalize_amount_unit(unit) for unit in have_unit]
    for accuracy, e in possible_entries:
        if "parsed" not in e:
            continue
        debug and print("considering", e["name"], e["Id"])
        want_unit = e["parsed"]["unit"]

        if want_unit in have_unit:
            debug and print("got direct match!", have_count, have_unit, have_count, want_unit)
            return (e, accuracy, have_count, want_unit, have_count, want_unit)
        for amount_name, amount_source, to_norm in e["parsed"]["amounts"]:
            debug and print(e["name"], "amount", amount_source)
            if amount_name in have_unit:
                debug and print("got match unit!", amount_source)
                return (e, accuracy, have_count, have_unit[0], have_count*to_norm, want_unit)
def parse_nutritional_values(nutri_values):
    def parse_german_float(f):
        return float(f.replace(",", "."))
    return {k: {"unit": v["Einheit"], "amount": parse_german_float(v["Menge"])} for k, v in nutri_values.items()}

def match_ingredient(ingredient):
    output = {"original": ingredient, "type": "ingredient" if "amount" in ingredient else "subtitle"}
    if output["type"] == "ingredient":
        amount_count, amount_type = parsed_amounts[ingredient["amount"]]

        if amount_count is None: # only happens when both amount_count and amount_type is None (else it is set to 1)
            #return cols + ["[ignored]"]
            output["matched"] = {"matched": False, "important": False, "message": "[unimportant]"}
            return output
            #amount_count = 1
            #amount_type = ["UNK"]
        if amount_type is None:
            amount_type = ["UNK", "Stück", "mittelgroß"]
        elif amount_type in ["dicke", "dicker"]:
            amount_type = [amount_type, "große", "großer"]
        else:
            amount_type = [amount_type]
        #print("[" + str(amount_count) + "*" + (amount_type)+ "] of ", end="")
        _ing_matches = all_ings_orig[ingredient["ingredient"]]
        ing_matches = [e for e in _ing_matches if e[1] > 0.85]
        if len(ing_matches) == 0:
            # not so great match but eh
            ing_matches = [_ing_matches[0]]


        debug and print("matches", ing_matches)
        debug and print("finding unit ", amount_type)
        match = match_amts(ing_matches=ing_matches, have_count=amount_count, have_unit=amount_type)

        if not match:
            message = "[no amount for match for [{}] on [{}]]".format("|".join(amount_type), "|".join(t for t, _ in ing_matches))
            output["matched"] = {"matched": False, "important": True, "message": message}
        else:
            fddb_entry, accuracy, count_weird, unit_weird, count_norm, unit_norm = match
            values = fddb_entry["parsed"]["for_100"]
            value_kcal_per_100 = float(values["Kalorien"]["Menge"].replace(",", "."))
            value_kcal = value_kcal_per_100 / 100 * count_norm
            output["matched"] = {
                "matched": True,
                "match_accuracy": accuracy,
                "weird": {"count": count_weird, "unit": unit_weird},
                "normal": {"count": count_norm, "unit": unit_norm},
                "name": fddb_entry["name"],
                "id": fddb_entry["Id"],
                "multiplier": count_norm / 100,
                "nutritional_values": values #parse_nutritional_values(values)
            }
    return output
def match_recipe(recipe):
    return {**recipe, "ingredients": [match_ingredient(ing) for ing in recipe["ingredients"]]}
    
def display_recipe(recipe):
    display(HTML("<h2>{}</h2>".format(escape(recipe["title"]))))
    ings = match_recipe(recipe)["ingredients"]
    #print(ings)
    def disp_ing(ing):
        if ing["type"] == "ingredient":
            cols = [ing["original"]["amount"], ing["original"]["ingredient"], "="]
            p = ing["matched"]
            if not p["matched"]:
                return cols + [p["message"]]
            
            weird = p["weird"]
            norm = p["normal"]
            if weird["unit"] == norm["unit"]:
                unittxt = "{}{}".format(norm["count"], norm["unit"])
            else:
                unittxt = "{} {} = {}{}".format(weird["count"], weird["unit"], norm["count"], norm["unit"])
            values = p["nutritional_values"]
            value_kcal_per_100 = float(values["Kalorien"]["Menge"].replace(",", "."))
            value_kcal = value_kcal_per_100 / 100 * norm["count"]
            cols.append("({:.0f}%) [{}] of {} = {:.0f} kcal".format(p["match_accuracy"] * 100, unittxt, p["name"], value_kcal))
            return cols
        elif ing["type"] == "subtitle":
            return ["->>" + ing["original"]["subtitle"]]
    ings_df = pd.DataFrame([disp_ing(ing) for ing in ings], columns=["inp amount", "inp ing", "", "paredd"])
    display(HTML(ings_df.to_html()))
                                      
for recipe in random.sample(recipes, 20):
    display_recipe(recipe)

,inp amount,inp ing,,paredd
0,30 g,"Steinpilze, getrocknet",=,(100%) [30g] of Steinpilze = 12 kcal
1,250 g,"Steinpilze, frische, möglichst kleine",=,(100%) [250g] of Steinpilze = 98 kcal
2,60 g,"Butter, oder Margarine",=,(100%) [60g] of Butter = 446 kcal
3,1,Zwiebel(n),=,"(96%) [1 UNK = 85.0g] of Zwiebel, frisch = 24 kcal"
4,½ TL,Thymian,=,[no amount for match for [Teelöffel] on [Thymian]]
5,etwas,Salz,=,[unimportant]
6,etwas,"Pfeffer, frisch gemahlen",=,[unimportant]
7,20 ml,Portwein,=,(100%) [20ml] of Portwein = 32 kcal
8,⅛ Liter,Kalbsfond,=,[no amount for match for [ml] on [Kalbsfond|Kalbsplätzli|Kalbskotelett|Kalbsfilet|Rinderfond|Kalbgulasch|Kalbfleischbällchen|Kalbspariser]]
9,200 g,Schlagsahne,=,(100%) [200g] of Schlagsahne = 574 kcal


,inp amount,inp ing,,paredd
0,3 cl,Gin,=,(100%) [30ml] of Gin = 72 kcal
1,1 ½ cl,Brandy (Cherry Brandy),=,(79%) [15.0ml] of Blanche = 7 kcal
2,1 Spritzer,Likör (Benedicitine D.O.M. oder jeder bessere Abteilikör),=,[no amount for match for [Spritzer] on [Schnaps|Likörkirschen]]
3,1 Spritzer,Cointreau,=,[no amount for match for [Spritzer] on [Cointreau]]
4,1 cl,Grenadine,=,[no amount for match for [ml] on [Mafaldine]]
5,12 cl,Ananassaft,=,(100%) [120ml] of Ananassaft = 60 kcal
6,1 ½ cl,"Limettensaft, frisch",=,(100%) [15.0ml] of Limettensaft = 5 kcal
7,1 Spritzer,Angostura,=,[no amount for match for [Spritzer] on [Diavolo]]
8,,Eis,=,[no amount for match for [UNK|Stück|mittelgroß] on [Eistee|Eiswürfel]]


,inp amount,inp ing,,paredd
0,1 kg,Hähnchenfleisch,=,(97%) [1000g] of Hähnchenhackfleisch = 1310 kcal
1,2 große,Zwiebel(n),=,"(96%) [2 große = 500.0g] of Zwiebel, frisch = 140 kcal"
2,200 ml,Schmand,=,(100%) [200ml] of Schmand = 488 kcal
3,etwas,Öl,=,[unimportant]
4,1 EL,Mehl,=,(90%) [1 Esslöffel = 10.0g] of Paniermehl = 35 kcal
5,2 Zehe/n,Knoblauch,=,"(100%) [2 Zehe = 6.0g] of Knoblauch, frisch = 9 kcal"
6,300 ml,Brühe oder Wasser,=,(90%) [300ml] of Klare Brühe = 9 kcal
7,etwas,Salz und Pfeffer,=,[unimportant]


,inp amount,inp ing,,paredd
0,500 g,Tomate(n),=,"(100%) [500g] of Tomaten, frisch = 90 kcal"
1,1 kleine,Gemüsezwiebel(n),=,[no amount for match for [kleine] on [Gemüsezwiebel|Putenzwiebelmett|Rstzwiebeln|Rinderzwiebelmett|Röstzwiebeln|Silberzwiebeln|Sielberzwiebeln|Zwiebelringe|Zwiebelsenf|Gemüsebouletten]]
2,100 g,"Speck (Südtiroler Speck), in Streifen geschnitten",=,(100%) [100g] of Speck = 271 kcal
3,2,"Peperoncini, getrocknet",=,[no amount for match for [UNK|Stück|mittelgroß] on [Peperoncini|Nudeln Maccheroncini]]
4,1 EL,Olivenöl,=,(100%) [1 Esslöffel = 10.0ml] of Olivenöl = 83 kcal
5,5 EL,Weißwein,=,"[no amount for match for [Esslöffel] on [Weißweinessig|Weißwein-Essig|Weißwein, lieblich|Weißwein, trocken|Weißweinschorle|Weißwein, Durchschnitt]]"
6,1,Lorbeerblatt,=,(82%) [1 UNK = 85.0g] of Laugenecke = 319 kcal
7,500 g,Spaghetti,=,(100%) [500g] of Spaghetti = 1750 kcal
8,1 EL,"Petersilie, frisch",=,"[no amount for match for [Esslöffel] on [Petersilie|Pesto Petersilie|Petersilie, getrocknet|Petersilie, frisch|Petersilie, gefriergetrocknet|Basilikum]]"
9,1 EL,Schnittlauch,=,[no amount for match for [Esslöffel] on [Schnittlauch|Schnittlauchring|Knabbeln|Reisflocken|Schalotte|Aufschnitt|Schnittlauchbutter|Knabbermäuschen|schnittlau topfen|Speisetopfen]]


,inp amount,inp ing,,paredd
0,50 g,Räuchertofu,=,(100%) [50g] of Räuchertofu = 106 kcal
1,50 g,Frischkäse natur,=,(100%) [50g] of Frischkäse natur = 126 kcal
2,4 Scheibe/n,"Aubergine(n) (ca. 0,5 cm dick)",=,[no amount for match for [Scheibe] on [Aubergine|Auberginenpüree|Auberginen-Creme|Auberginen-Nuggets|Auberginencreme|Gebratene Auberginen]]
3,1,Tomate(n),=,"(100%) [1 UNK = 130.0g] of Tomaten, frisch = 23 kcal"
4,2,Brötchen (Dinkelvollkornbrötchen),=,(95%) [2 UNK = 140.0g] of Laugen Brötchen = 388 kcal
5,,Olivenöl,=,(100%) [1 UNK = 20.0ml] of Olivenöl = 166 kcal
6,,Salz und Pfeffer aus der Mühle,=,[no amount for match for [UNK|Stück|mittelgroß] on [Zöpfli mit Salz und Kümmel]]
7,4 Blätter,"Basilikum, frisches",=,"[no amount for match for [Blätter] on [Basilikum|Erbse Basilikum|Tomate Basilikum|Basilikumpesto|Basilikumöl|Basilikum Salatsoe|Basilikumcreme|Basilikum, getrocknet|Tomate-Basilikum|Sauce Basilikum]]"


...

,inp amount,inp ing,,paredd
0,125 g,Butter,=,(100%) [125g] of Butter = 929 kcal
1,200 ml,"Milch (Bananen-), Fertigprodukt",=,(100%) [200ml] of Milch = 110 kcal
2,100 g,Zucker,=,(100%) [100g] of Zucker = 400 kcal
3,1 Tüte/n,Vanillinzucker,=,(100%) [1 Tüte = 8.0g] of Vanillinzucker = 32 kcal
4,500 g,Mehl,=,(100%) [500g] of Mehl = 1740 kcal
5,2 m.-große,Ei(er),=,"(100%) [2 mittelgroß = 110.0g] of Ei, vom Huhn = 151 kcal"
6,1 Würfel,Hefe,=,"(88%) [1 Würfel = 42.0g] of Hefe, frisch = 35 kcal"
7,1 Prise(n),Salz,=,(100%) [1 Prise = 1.0g] of Salz = 0 kcal


,inp amount,inp ing,,paredd
0,100 g,Frischkäse (4 % Fett absolut),=,(100%) [100g] of Frischkäse = 200 kcal
1,200 ml,"Schlagsahne, frische",=,(100%) [200ml] of Schlagsahne = 584 kcal
2,80 g,Kekse (Amarettini),=,(100%) [80g] of Kekse = 54 kcal
3,1 Pck.,Vanillezucker,=,(100%) [1 Packung = 8.0g] of Vanillezucker = 32 kcal
4,2 Tasse/n,Espresso,=,"[no amount for match for [Tasse] on [Espresso|Esspresso|Typ Espresso|Espresso doppio|Latte Espresso|Soypresso|Praline, Espresso|Espresso Macchiato|Espresso Cappuccino]]"
5,75 ml,Amaretto,=,[no amount for match for [ml] on [Amaretti|Latte Amaretto|Amarettini]]
6,4 Prisen,Kakaopulver,=,[no amount for match for [Prisen] on [Kakaopulver|Back Kakaopulver|Kakao Pulver|rohes Kakaopulver|Kakaopulver (neu)|Kakaomandeln|Kakaoschaum-Törtchen|Kakaospitzen|Kakao Butter|Kakaopulver Dukan]]
7,n. B.,Schokodekor nach Wahl,=,[unimportant]
8,evtl.,"Obst, frisches nach Saison",=,[unimportant]


,inp amount,inp ing,,paredd
0,115 g,Butter,=,(100%) [115g] of Butter = 854 kcal
1,450 g,"Zucker, braun",=,(100%) [450g] of Zucker = 1800 kcal
2,120 ml,Milch,=,(100%) [120ml] of Milch = 66 kcal
3,200 g,Erdnussbutter,=,(100%) [200g] of Erdnussbutter = 1240 kcal
4,,Vanille - Extrakt oder Vanillezucker,=,[no amount for match for [UNK|Stück|mittelgroß] on [Vanille-Extrakt]]
5,400 g,Puderzucker,=,(100%) [400g] of Puderzucker = 1600 kcal


,inp amount,inp ing,,paredd
0,125 g,Margarine,=,(100%) [125g] of Margarine = 900 kcal
1,100 g,Zucker,=,(100%) [100g] of Zucker = 400 kcal
2,1 Prise(n),Salz,=,(100%) [1 Prise = 1.0g] of Salz = 0 kcal
3,2,Ei(er),=,"(100%) [2 UNK = 110.0g] of Ei, vom Huhn = 151 kcal"
4,175 g,Mehl,=,(100%) [175g] of Mehl = 609 kcal
5,1 TL,Backpulver,=,(100%) [1 Teelöffel = 3.0g] of Backpulver = 3 kcal
6,400 g,Zwetschgen,=,"(97%) [400g] of Zwetschge, frisch = 172 kcal"
7,,Puderzucker zum Bestäuben,=,(94%) [1 UNK = 10.0g] of Palmzucker = 40 kcal
8,,Fett für die Form,=,[no amount for match for [UNK|Stück|mittelgroß] on [Fit fürs Leben]]


,inp amount,inp ing,,paredd
0,1 kg,"Kartoffel(n), festkochende Sorte",=,(100%) [1000g] of Kartoffeln = 770 kcal
1,250 ml,"Fleischbrühe, heiße",=,(96%) [250ml] of Rinderbrühe = 68 kcal
2,4 EL,"Essig, (Kräuteressig)",=,(100%) [4 Esslöffel = 40.0ml] of Essig = 8 kcal
3,300 g,"Hähnchenbrustfilet(s), in kleine Würfel geschnitten",=,(100%) [300g] of Hähnchenbrustfilets = 291 kcal
4,1,"Zwiebel(n), fein gehackt",=,"(96%) [1 UNK = 85.0g] of Zwiebel, frisch = 24 kcal"
5,->>Für die Sauce:,None,None,None
6,1 TL,"Senf, idealerweise Estragonsenf",=,"(92%) [1 Teelöffel = 10.0g] of Senf, Scharf = 13 kcal"
7,2 EL,Mayonnaise,=,(100%) [2 Esslöffel = 60.0g] of Mayonnaise = 432 kcal
8,2 EL,Naturjoghurt,=,"[no amount for match for [Esslöffel] on [Naturjoghurt|Naturjoghurt leicht|Naturjoghurt, Natur|naturjoghurd|Naturell|Feinjoghurt|Natur-Reis]]"
9,4 EL,"Essig, idealerweise Estragonessig",=,(100%) [4 Esslöffel = 40.0ml] of Essig = 8 kcal


In [61]:
with open("data/recipes/recipes_matched.jsonl", "w") as f:
    for recipe in tqdm(recipes):
        json.dump(match_recipe(recipe), f)
        f.write("\n")

In [ ]:
fddb_entries["Ei, vom Huhn"]

In [31]:
normalize_out_ingredient("Ei, vom Huhn")

['Ei', 'Ei, vom Huhn', 'Eier']

In [32]:
match_ingredient({"amount": "1 Pck.", "ingredient": "Backpulver"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/ja_backpulver/index.html',
  'match_accuracy': 1.0000001192092896,
  'matched': True,
  'multiplier': 0.15,
  'name': 'Backpulver',
  'normal': {'count': 15.0, 'unit': 'g'},
  'nutritional_values': {'Brennwert': {'Einheit': 'kJ', 'Menge': '339'},
   'Broteinheiten': {'Einheit': '', 'Menge': '1,6'},
   'Fett': {'Einheit': 'gBroteinheiten', 'Menge': '0,11,6'},
   'Kalorien': {'Einheit': 'kcal', 'Menge': '81'},
   'Kohlenhydrate': {'Einheit': 'g', 'Menge': '19,7'},
   'Protein': {'Einheit': 'g', 'Menge': '0,1'},
   'davonZucker': {'Einheit': 'g', 'Menge': '0'}},
  'weird': {'count': 1, 'unit': 'Packung'}},
 'original': {'amount': '1 Pck.', 'ingredient': 'Backpulver'},
 'type': 'ingredient'}

In [33]:
z = fddb_entries["Backpulver"][2]["parsed"]["for_100"]["Kalorien"]["Menge"]
float(z.replace(",", "."))

100.0

In [34]:
match_ingredient({"amount": "1", "ingredient": "Schalotte(n)"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/merkur_maisecke/index.html',
  'match_accuracy': 0.8587009310722351,
  'matched': True,
  'multiplier': 1.2,
  'name': 'Maisecke',
  'normal': {'count': 120.0, 'unit': 'g'},
  'nutritional_values': {'Brennwert': {'Einheit': 'kJ', 'Menge': '1411'},
   'Broteinheiten': {'Einheit': '', 'Menge': '2,9'},
   'Fett': {'Einheit': 'gBroteinheiten', 'Menge': '182,9'},
   'Kalorien': {'Einheit': 'kcal', 'Menge': '337'},
   'Kohlenhydrate': {'Einheit': 'gFettgBroteinheiten', 'Menge': '35182,9'},
   'Protein': {'Einheit': 'g', 'Menge': '7'}},
  'weird': {'count': 1, 'unit': 'UNK'}},
 'original': {'amount': '1', 'ingredient': 'Schalotte(n)'},
 'type': 'ingredient'}

In [35]:
match_ingredient({"amount": "1 Glas", "ingredient": "Schattenmorellen"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/schneekoppe_schattenmorellen/index.html',
  'match_accuracy': 0.9999998807907104,
  'matched': True,
  'multiplier': 3.4,
  'name': 'Schattenmorellen',
  'normal': {'count': 340.0, 'unit': 'g'},
  'nutritional_values': {'Ballaststoffe': {'Einheit': 'g', 'Menge': '0'},
   'Brennwert': {'Einheit': 'kJ', 'Menge': '176'},
   'Broteinheiten': {'Einheit': '', 'Menge': '0,7'},
   'Fett': {'Einheit': 'g', 'Menge': '0,3'},
   'Kalorien': {'Einheit': 'kcal', 'Menge': '42'},
   'Kohlenhydrate': {'Einheit': 'g', 'Menge': '8,4'},
   'Protein': {'Einheit': 'g', 'Menge': '0,4'},
   'davonZucker': {'Einheit': 'g', 'Menge': '8,4'}},
  'weird': {'count': 1, 'unit': 'Glas'}},
 'original': {'amount': '1 Glas', 'ingredient': 'Schattenmorellen'},
 'type': 'ingredient'}

In [36]:
match_ingredient({"amount": "3 m.-große", "ingredient": "Ei(er)"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/naturprodukt_ei_vom_huhn/index.html',
  'match_accuracy': 0.9999999403953552,
  'matched': True,
  'multiplier': 1.65,
  'name': 'Ei, vom Huhn',
  'normal': {'count': 165.0, 'unit': 'g'},
  'nutritional_values': {'Ballaststoffe': {'Einheit': 'g', 'Menge': '0'},
   'Brennwert': {'Einheit': 'kJ', 'Menge': '572'},
   'Broteinheiten': {'Einheit': '', 'Menge': '0,1'},
   'Cholesterin': {'Einheit': 'mgWassergehalt%', 'Menge': '39676%'},
   'Fett': {'Einheit': 'g', 'Menge': '9,3'},
   'Kalorien': {'Einheit': 'kcal', 'Menge': '137'},
   'Kohlenhydrate': {'Einheit': 'g', 'Menge': '1,5'},
   'Protein': {'Einheit': 'g', 'Menge': '11,9'},
   'davonZucker': {'Einheit': 'g', 'Menge': '1,5'}},
  'weird': {'count': 3, 'unit': 'mittelgroß'}},
 'original': {'amount': '3 m.-große', 'ingredient': 'Ei(er)'},
 'type': 'ingredient'}

In [37]:
match_ingredient({"amount": "", "ingredient": "Olivenöl"})

{'matched': {'id': 'https://fddb.info/db/de/lebensmittel/minato_olivenoel/index.html',
  'match_accuracy': 0.9999998807907104,
  'matched': True,
  'multiplier': 0.2,
  'name': 'Olivenöl',
  'normal': {'count': 20.0, 'unit': 'ml'},
  'nutritional_values': {'Ballaststoffe': {'Einheit': 'g', 'Menge': '0'},
   'Brennwert': {'Einheit': 'kJ', 'Menge': '3466'},
   'Broteinheiten': {'Einheit': '', 'Menge': '0'},
   'Cholesterin': {'Einheit': 'mg', 'Menge': '0'},
   'Fett': {'Einheit': 'g', 'Menge': '92'},
   'Kalorien': {'Einheit': 'kcal', 'Menge': '828'},
   'Kohlenhydrate': {'Einheit': 'g', 'Menge': '0'},
   'Protein': {'Einheit': 'g', 'Menge': '0'},
   'davonZucker': {'Einheit': 'g', 'Menge': '0'}},
  'weird': {'count': 1, 'unit': 'UNK'}},
 'original': {'amount': '', 'ingredient': 'Olivenöl'},
 'type': 'ingredient'}